In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz 
import os
import json
from pathlib import Path
import spacy
import en_core_web_sm
import nltk
import re
nlp = en_core_web_sm.load()
nlp.max_length = 6000000 

In [2]:
# Importing environmental variables library that reads from the .env file
from dotenv import load_dotenv

# Loading key-value pairs from the .env file into the OS environment
load_dotenv()

# Reading the key-value pairs from the OS environment
user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
db_hostname = os.getenv("DB_HOST")
db_name = os.getenv("DB_DATABASE")

# Using those variables in the connection string using "F" strings
conn_string = f"mysql+mysqldb://{user}:{password}@{db_hostname}/{db_name}?charset=utf8mb4"
engine = create_engine(conn_string)

In [3]:
def lemmaspacy(my_new_str):
    my_new_str = re.sub(' +', ' ', my_new_str)
    doc = nlp(my_new_str)
    return " ".join([token.lemma_ for token in doc]) # joining all the word tokens after lemmatizer implementation

In [4]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmawordnet(my_new_str):
    sentence_words = nltk.word_tokenize(my_new_str)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(word, pos='n') for word in sentence_words]) 
    return lemmatized_output

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\t1nipun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
def clean_dataframe(dataframe, column_name):
    dataframe[column_name] = dataframe[column_name].str.lower()
    pattern = '|'.join(['<s>', '</s>',])
    dataframe[column_name] = dataframe[column_name].str.replace(pattern, '')
    dataframe[column_name] = dataframe[column_name].str.replace('\(s\)', 's') #reference: https://stackoverflow.com/questions/51440233/how-to-remove-the-values-which-are-in-parentheses-in-pandashttps://stackoverflow.com/questions/51440233/how-to-remove-the-values-which-are-in-parentheses-in-pandas
    return dataframe    

In [6]:
def clean_header(my_str):
    #pattern = '|'.join(['<s>', '</s>',])
    my_str = my_str.replace('<s>', '')
    my_str = my_str.replace('</s>', '')
    my_str = my_str.replace('(s)', 's') #reference: https://stackoverflow.com/questions/51440233/how-to-remove-the-values-which-are-in-parentheses-in-pandashttps://stackoverflow.com/questions/51440233/how-to-remove-the-values-which-are-in-parentheses-in-pandas
    return my_str  

In [7]:
query = "SELECT * FROM flat_data;"

with engine.connect() as conn:
    df = pd.read_sql(query, conn)
    #clean_dataframe(df, 'table_content')
dic = {}

for row in df.itertuples():
    # converting JSON string to a list of lists of strings
    table = json.loads(row.table_content)
    headers = table[0]  # column headers  
    for header in headers:
        #header = lemmawordnet(header)
        #header = clean_header(header)
        #header = lemmaspacy(header)
        #header = re.sub(r'\(.*?\)', lambda x: ''.join(x.group(0).split()), header) # removing whitespace between parentheses (reference: https://stackoverflow.com/questions/34088489/how-to-remove-whitespace-inside-brackethttps://stackoverflow.com/questions/34088489/how-to-remove-whitespace-inside-bracket)
        header = " ".join(header.split())
        if header in dic:
            dic[header] += 1
        else:
            dic[header] = 1

my_list = [(header, count) for header, count in dic.items()]  # Converting to list
my_list.sort(key=lambda tup: tup[1], reverse=True)  # sorting the list

total_headers = 0
print(f"COUNT\tHEADER")
for item in my_list:
    total_headers += item[1]
    print(f"{item[1]}\t{item[0]}")
print()
print(f'Total headers: {total_headers}; unique headers: {len(my_list)}')

COUNT	HEADER
68	Land Use
58	Status
55	Location
55	VEC
55	GIS
55	Topic
51	Legal Location
41	Notes
40	Recommendation(s)
31	Environmental Issues
24	Cover Type
23	Cover Ranking Assignment Low
23	Cover Ranking Assignment Moderate
23	Cover Ranking Assignment High
22	KP
19	Current Status 2011
18	1<s>st</s> Year Issue/ Condition
18	Recommendations
18	Environmental Issue/ Concern
18	Current Status 2012
16	Legal Location (W4M)
16	Current Status 2013
15	Issue(s)
15	Current Status 2010
15	Recommended Measure(s) to Resolve Issue(s) and Schedule
14	1<s>st</s> Year (2014) Issue(s)
14	3<s>rd</s> Year (2016) PCRM Assessment and Work Completed
14	Legal Land Description
14	Environmental Issue(s)
13	2nd Year (2014) PCRM Assessment and Work Completed
13	3rd Year (2015) PCRM Assessment and Work Completed
13	Surface Drainage (ponding)
13	Condition/ Issue Rating
13	Legal Location (W6M)
13	Approximate KP
13	Alignment Sheet
12	4th Year (2016) PCRM Assessment and Work Completed
12	Potential Adverse Environmental

In [363]:
df3 = pd.DataFrame(my_list)
df3.to_csv('headers4.csv', encoding = 'utf-8-sig', index = False)

In [ ]:
for row in df.itertuples():
    table = json.loads(row.table_content)
    print(type(table))
    lst_element1 = "SPREAD"
    lst_element2 = "QUARTER SECTION"
    if lst_element1 in table[0][0] and lst_element2 in table[0][1]:
        print(row.tableId)

In [370]:
for row in df.itertuples():
    table = json.loads(row.table_content)
    if table[0][len(table[0])-1] == "Topic":
        data = pd.DataFrame(table)
        data.to_csv(r"C:\Users\t1nipun\Desktop\PCMR\human-robot\Data_Analysis\csvs\\" + row.tableId + '.csv', encoding = 'utf-8-sig', index = False, header = None)
    else:
        table[0].extend(['VEC', 'GIS', 'Topic'])
        for i in range(1, len(table)-1):
            table[i].extend(['','',''])
        data = pd.DataFrame(table)
        data.to_csv(r"C:\Users\t1nipun\Desktop\PCMR\human-robot\Data_Analysis\csvs\\" + row.tableId + '.csv', encoding = 'utf-8-sig', index = False, header = None)